# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
# Capitalizing the City Name
city_data_df.City = city_data_df.City.str.title()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Adamstown,-25.0660,-130.1015,21.43,56,81,8.29,PN,1712856692
1,1,Waitangi,-43.9535,-176.5597,16.01,95,55,3.13,NZ,1712856692
2,2,Sao Felix Do Xingu,-6.6447,-51.9950,29.06,85,44,0.88,BR,1712856693
3,3,Port-Aux-Francais,-49.3500,70.2167,6.14,96,98,6.48,TF,1712856693
4,4,Stanley,54.8680,-1.6985,14.29,78,20,5.66,GB,1712856615


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"    
)
# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Adamstown,-25.0660,-130.1015,21.43,56,81,8.29,PN,1712856692
1,1,Waitangi,-43.9535,-176.5597,16.01,95,55,3.13,NZ,1712856692
2,2,Sao Felix Do Xingu,-6.6447,-51.9950,29.06,85,44,0.88,BR,1712856693
3,3,Port-Aux-Francais,-49.3500,70.2167,6.14,96,98,6.48,TF,1712856693
4,4,Stanley,54.8680,-1.6985,14.29,78,20,5.66,GB,1712856615
...,...,...,...,...,...,...,...,...,...,...
579,579,Kurchatov,51.6601,35.6521,21.34,75,100,3.56,RU,1712856901
580,580,Shiraoi,42.5500,141.3500,7.10,79,100,1.16,JP,1712856901
581,581,Saint-Philippe,-21.3585,55.7679,26.77,86,80,2.05,RE,1712856902
582,582,Cooper'S Town,26.8667,-77.5167,23.63,90,39,11.08,BS,1712856902


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Temperate/warm destinations (>20 max temp) and not-so-cloudy (<50% clouds)
city_data_df.dropna(axis=0,how="any",inplace=True)
ideal_df = city_data_df.copy()
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] >= 20) & (ideal_df["Cloudiness"]<=50)]
# Drop any rows with null values
ideal_df.dropna(axis=0,how="any",inplace=True)
# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Sao Felix Do Xingu,-6.6447,-51.9950,29.06,85,44,0.88,BR,1712856693
25,25,Kailua-Kona,19.6406,-155.9956,25.89,86,20,0.00,US,1712856478
29,29,Big Spring,32.2504,-101.4787,21.25,29,0,4.12,US,1712856702
32,32,Nova Sintra,14.8667,-24.7167,22.86,73,6,6.42,CV,1712856703
46,46,Avarua,-21.2078,-159.7750,27.03,89,20,6.17,CK,1712856708
...,...,...,...,...,...,...,...,...,...,...
570,570,Medina,24.4686,39.6142,30.24,8,0,3.09,SA,1712856602
574,574,Haiku-Pauwela,20.9219,-156.3051,23.36,82,20,4.92,US,1712856799
577,577,Alto Barinas,8.5931,-70.2261,37.37,25,27,4.39,VE,1712856900
578,578,Vung Tau,10.3460,107.0843,27.57,82,18,10.86,VN,1712856900


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City","Lat","Lng","Humidity","Country"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
2,Sao Felix Do Xingu,-6.6447,-51.9950,85,BR,
25,Kailua-Kona,19.6406,-155.9956,86,US,
29,Big Spring,32.2504,-101.4787,29,US,
32,Nova Sintra,14.8667,-24.7167,73,CV,
46,Avarua,-21.2078,-159.7750,89,CK,
...,...,...,...,...,...,...
570,Medina,24.4686,39.6142,8,SA,
574,Haiku-Pauwela,20.9219,-156.3051,82,US,
577,Alto Barinas,8.5931,-70.2261,25,VE,
578,Vung Tau,10.3460,107.0843,82,VN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
hotel_category = "accommodation.hotel"
params = {
    "apiKey": f"{geoapify_key}",
    "categories": f"{hotel_category}",
    "limit": f"{limit}"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    circle_lon = hotel_df.loc[index,"Lng"]
    circle_lat = hotel_df.loc[index,"Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{circle_lon},{circle_lat},{radius}"
    params["bias"] = f"proximity:{circle_lon},{circle_lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params).json()
    
 

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Sao Felix Do Xingu - nearest hotel: Hotel Terraço
Kailua-Kona - nearest hotel: Kona Seaside Hotel
Big Spring - nearest hotel: Settles Hotel
Nova Sintra - nearest hotel: Residência Ka Dencho
Avarua - nearest hotel: Paradise Inn
Nova Vicosa - nearest hotel: Pousada Pontal da Barra
Cockburn Town - nearest hotel: The Salt Raker Inn
Beau Vallon - nearest hotel: Le Jardin de Beau Vallon
Atar - nearest hotel: Residence Egueil
Arraial Do Cabo - nearest hotel: No hotel found
El Granada - nearest hotel: Beach House
Ciudad Lazaro Cardenas - nearest hotel: Hotel Sol del Pacífico
Aswan - nearest hotel: Oscar Hotel
Bel Ombre - nearest hotel: Villa La Cachette
Myeik - nearest hotel: Green Eyes Hotel
Port Douglas - nearest hotel: Mantra Aqueous Port Douglas
West Island - nearest hotel: Cocos Village Bungalows
Georgetown - nearest hotel: Page 63 hostel
San Patricio - nearest hotel: Best Western Plus
Ciudad Del Maiz - nearest hotel: Huasteca Express
Al Jawf - nearest hotel: No hote

,City,Lat,Lng,Humidity,Country,Hotel Name
2,Sao Felix Do Xingu,-6.6447,-51.9950,85,BR,Hotel Terraço
25,Kailua-Kona,19.6406,-155.9956,86,US,Kona Seaside Hotel
29,Big Spring,32.2504,-101.4787,29,US,Settles Hotel
32,Nova Sintra,14.8667,-24.7167,73,CV,Residência Ka Dencho
46,Avarua,-21.2078,-159.7750,89,CK,Paradise Inn
...,...,...,...,...,...,...
570,Medina,24.4686,39.6142,8,SA,The Oberoi Madina
574,Haiku-Pauwela,20.9219,-156.3051,82,US,Inn At Mama's Fish House
577,Alto Barinas,8.5931,-70.2261,25,VE,Hotel Turístico Varyná
578,Vung Tau,10.3460,107.0843,82,VN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot


hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "Country",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)